# Ham or Spam?

In [47]:
# when installing nltk for the first time we need to also download a few built in libraries
!pip install nltk
import nltk
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# stematisation : 
from nltk.stem.porter import PorterStemmer 

# bags of modeling
from sklearn.feature_extraction.text import CountVectorizer

# accuray 


from sklearn.naive_bayes import MultinomialNB

import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_pickle("email")

df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


The dataset is made up of email that are classified as ham [0] or spam[1]. You need to clean the dataset before training a prediction model.

## Remove Punctuation

👇 Create a function to remove the punctuation. Apply it to the entire data and add the output as a new column in the dataframe called `clean_text`

In [3]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
for punctuation in string.punctuation : 
    df['text'] = df['text'].str.replace(punctuation, "")
    
df

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_7712\1551738433.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace(punctuation, "")


,text,spam
0,Subject naturally irresistible your corporate ...,1
1,Subject the stock trading gunslinger fanny is...,1
2,Subject unbelievable new homes made easy im w...,1
3,Subject 4 color printing special request addi...,1
4,Subject do not have money get software cds fr...,1
...,...,...
5723,Subject re research and development charges t...,0
5724,Subject re receipts from visit jim thanks ...,0
5725,Subject re enron case study update wow all ...,0
5726,Subject re interest david please call shi...,0


## Lower Case

👇 Create a function to lower case the text. Apply it to `clean_text`

In [5]:
def lower(clean_text) :
    clean_text= clean_text.str.lower()
    return clean_text

In [6]:
df['text'] = lower(df['text'])
df

,text,spam
0,subject naturally irresistible your corporate ...,1
1,subject the stock trading gunslinger fanny is...,1
2,subject unbelievable new homes made easy im w...,1
3,subject 4 color printing special request addi...,1
4,subject do not have money get software cds fr...,1
...,...,...
5723,subject re research and development charges t...,0
5724,subject re receipts from visit jim thanks ...,0
5725,subject re enron case study update wow all ...,0
5726,subject re interest david please call shi...,0


## Remove Numbers

👇 Create a function to remove numbers from the text. Apply it to `clean_text`

In [7]:
def numbers(clean_text):
    clean_text = clean_text.str.replace(r'\b\d+\b', '')
    return clean_text

In [8]:
df['text'] = numbers(df['text'])
df

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_7712\2144780378.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_text = clean_text.str.replace(r'\b\d+\b', '')


,text,spam
0,subject naturally irresistible your corporate ...,1
1,subject the stock trading gunslinger fanny is...,1
2,subject unbelievable new homes made easy im w...,1
3,subject color printing special request addit...,1
4,subject do not have money get software cds fr...,1
...,...,...
5723,subject re research and development charges t...,0
5724,subject re receipts from visit jim thanks ...,0
5725,subject re enron case study update wow all ...,0
5726,subject re interest david please call shi...,0


## Remove StopWords

👇 Create a function to remove stopwords from the text. Apply it to `clean_text`.

In [11]:
def tokenize(clean_text) :
    """nltk le fait automatiquement : il prend la string et la sépare en mots"""
    return word_tokenize(clean_text)

In [13]:
def remov(clean_text): 
    """prends ma liste de stopwords"""
    stop_words = set(stopwords.words('english'))
# conserve les mots si ils ne sont pas dans la liste du dessus
    words = [w for w in clean_text if w not in stop_words]
# prend une liste de string et la remets en une liste avec tous les mots 
    words = ' '.join(words)
    return words

In [14]:
# apply prend la fonction l'applique à chaque ligne de la colone 
df['clean_text']= df['text'].apply(tokenize)
df['clean_text'] = df['clean_text'].apply(remov)
df

,text,spam,clean_text
0,subject naturally irresistible your corporate ...,1,subject naturally irresistible corporate ident...
1,subject the stock trading gunslinger fanny is...,1,subject stock trading gunslinger fanny merrill...
2,subject unbelievable new homes made easy im w...,1,subject unbelievable new homes made easy im wa...
3,subject color printing special request addit...,1,subject color printing special request additio...
4,subject do not have money get software cds fr...,1,subject money get software cds software compat...
...,...,...,...
5723,subject re research and development charges t...,0,subject research development charges gpg forwa...
5724,subject re receipts from visit jim thanks ...,0,subject receipts visit jim thanks invitation v...
5725,subject re enron case study update wow all ...,0,subject enron case study update wow day super ...
5726,subject re interest david please call shi...,0,subject interest david please call shirley cre...


## Lemmatize

👇 Create a function to lemmatize the text. Make sure the output is a single string, not a list of words. Apply it to `clean_text`.

In [19]:
def stemmatize(clean_text) :
    """cette fonction stemmatize la colone de notre choix"""
    stemmer  = PorterStemmer()
    stemmed = [stemmer.stem(i) for i in clean_text]
    return clean_text

In [20]:
df['clean_text'] = stemmatize(df['clean_text'])
df

,text,spam,clean_text
0,subject naturally irresistible your corporate ...,1,subject naturally irresistible corporate ident...
1,subject the stock trading gunslinger fanny is...,1,subject stock trading gunslinger fanny merrill...
2,subject unbelievable new homes made easy im w...,1,subject unbelievable new homes made easy im wa...
3,subject color printing special request addit...,1,subject color printing special request additio...
4,subject do not have money get software cds fr...,1,subject money get software cds software compat...
...,...,...,...
5723,subject re research and development charges t...,0,subject research development charges gpg forwa...
5724,subject re receipts from visit jim thanks ...,0,subject receipts visit jim thanks invitation v...
5725,subject re enron case study update wow all ...,0,subject enron case study update wow day super ...
5726,subject re interest david please call shi...,0,subject interest david please call shirley cre...


## Bag-of-words Modelling

👇 Vectorize the `clean_text` to a Bag-of-Words representation with a default CountVectorizer . Save as `X_bow`.

In [45]:
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(df['clean_text'])
print(X_bow)

  (0, 28685)	1
  (0, 19719)	1
  (0, 15195)	1
  (0, 6423)	1
  (0, 14050)	1
  (0, 17491)	1
  (0, 24333)	1
  (0, 13028)	1
  (0, 24457)	1
  (0, 5654)	3
  (0, 18005)	2
  (0, 11785)	1
  (0, 28949)	1
  (0, 14592)	1
  (0, 15240)	1
  (0, 12444)	2
  (0, 4412)	1
  (0, 17303)	3
  (0, 28666)	1
  (0, 28315)	1
  (0, 21188)	1
  (0, 32368)	2
  (0, 17782)	2
  (0, 29384)	1
  (0, 19391)	2
  :	:
  (5727, 12568)	1
  (5727, 19161)	1
  (5727, 4279)	1
  (5727, 21168)	2
  (5727, 17975)	1
  (5727, 2090)	9
  (5727, 27281)	1
  (5727, 9695)	1
  (5727, 9484)	1
  (5727, 13742)	1
  (5727, 10487)	1
  (5727, 16624)	1
  (5727, 28174)	1
  (5727, 9485)	1
  (5727, 16818)	1
  (5727, 31633)	1
  (5727, 4987)	1
  (5727, 26429)	1
  (5727, 9879)	5
  (5727, 32528)	1
  (5727, 26634)	1
  (5727, 13944)	1
  (5727, 30270)	1
  (5727, 6731)	1
  (5727, 7744)	1


👇 Cross-validate a MultinomialNB model with the Bag-of-words. Score the model's accuracy.

In [48]:
nb_model = MultinomialNB()
nb_model.fit(X_bow, df['spam'])
nb_model.score(X_bow, df['spam'])

0.9965083798882681

⚠️ Please push the exercise once you are done 🙃

## 🏁 